In [ ]:
import tensorflow as tf
from tensorflow_addons.callbacks import AverageModelCheckpoint
import tensorflow_hub as hub

In [ ]:
from tflite_model_maker import object_detector
from pathlib import Path

voc_train_ds_path = Path("./data/annotations-train/watch-faces-train-PascalVOC-export")
voc_val_ds_path = Path(
    "./data/annotations-validation/watch-faces-validation-PascalVOC-export")
train_data = object_detector.DataLoader.from_pascal_voc(
    str(voc_train_ds_path / 'JPEGImages'),
    str(voc_train_ds_path / 'Annotations'),
    label_map={1: 'watch-face'},
        max_num_images=2
)
validation_data = object_detector.DataLoader.from_pascal_voc(
    str(voc_val_ds_path / 'JPEGImages'),
    str(voc_val_ds_path / 'Annotations'),
    label_map={1: 'watch-face'},
                        max_num_images=1


)
len(train_data), len(validation_data)

In [ ]:
ds =  train_data.gen_dataset(spec)

from PIL import Image
import matplotlib.pyplot as plt
for x in ds.take(1).as_numpy_iterator():
    # print(x)
    img = x[0][0]
    print(img.shape)
    plt.imshow(img)
    # Image.fromarray(img)

In [ ]:
from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras.train_lib import EfficientDetNetTrain
from tensorflow_examples.lite.model_maker.core.task.model_spec.object_detector_spec import EfficientDetModelSpec
from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras import efficientdet_keras
import tensorflow as tf
from tflite_model_maker import model_spec
spec = model_spec.get('efficientdet_lite0')
dict(spec.config)

In [ ]:
spec.config.update({'num_classes':1, 'weight_decay': 0, 'image_size': (224, 224)})
# temporary disable augmentations
spec.config.update({'jitter_max': 1.0, 'jitter_min': 1.0 })
class MyModel(EfficientDetNetTrain):

    @classmethod
    def conv2d_layer(cls):
        data_format = 'channels_last'
        config_separable_conv = True
        return efficientdet_keras.ClassNet.conv2d_layer(
            config_separable_conv, data_format)
    def _backbone_base_model(self):
        backbone = tf.keras.applications.ResNet50(
        include_top=False, input_shape=[None, None, 3]
        )
        c3_output, c4_output, c5_output = [
            backbone.get_layer(layer_name).output
            for layer_name in ["conv3_block4_out", "conv4_block6_out", "conv5_block3_out"]
        ]
        return tf.keras.Model(
            inputs=[backbone.inputs], outputs=[c3_output, c4_output, c5_output]
        )


        return tf.keras.models.Sequential(
                [
                    self.conv2d_layer()(kernel_size=3, filters=self.num_filters, activation=tf.nn.relu6, padding='same'),
                    tf.keras.layers.MaxPool2D(),
                    self.conv2d_layer()(kernel_size=3, filters=self.num_filters*2, activation=tf.nn.relu6,padding='same'),
                    tf.keras.layers.MaxPool2D(),
                    self.conv2d_layer()(kernel_size=3, filters=self.num_filters*2, activation=tf.nn.relu6,padding='same'),
                    tf.keras.layers.MaxPool2D(),
                 ]
        )


    def __init__(self):
        super(MyModel, self).__init__(config=spec.config)
        data_format = 'channels_last'
        config_separable_conv = True
        self.num_filters = 256
        config_aspect_ratios = spec.config.aspect_ratios
        config_num_scales = spec.config.num_scales
        config_num_classes = spec.config.num_classes
        num_anchors = len(config_aspect_ratios) * config_num_scales

        self.base_model = self._backbone_base_model()

        self.pre_classes = self.conv2d_layer()(kernel_size=3, filters=self.num_filters, activation=tf.nn.relu6,padding='same')
        self.pre_box = self.conv2d_layer()(kernel_size=3, filters=self.num_filters, activation=tf.nn.relu6,padding='same')


        self.classes = efficientdet_keras.ClassNet.classes_layer(
            self.conv2d_layer(),
            config_num_classes,
            num_anchors,
            name='class_net/class-predict',
        )

        self.boxes = efficientdet_keras.BoxNet.boxes_layer(
            config_separable_conv,
            num_anchors,
            data_format,
            name='box_net/box-predict')
    # log_dir = os.path.join(self.config.model_dir, 'train_images')
    # self.summary_writer = tf.summary.create_file_writer(log_dir)

    def call(self, inputs, training):
        # cls_outputs, box_outputs = self.base_model(inputs, training=training)
        outputs = self.base_model(inputs, training=training)

        # cls_outputs = [self.classes(self.pre_classes(outputs))]
        # box_outputs = [self.boxes(self.pre_box(outputs))]
        config_max_level = 7
        self_config_min_level = 3
        cls_outputs = [None for i in range(2)]
        box_outputs = [None for i in range(2)]
        for i in range(2):
            cls_outputs[i] = self.classes(outputs[i])
            box_outputs[i] = self.boxes(outputs[i])
            # outputs[i] = self.classes(outputs[i])
            # outputs[i] = self.boxes(outputs[i])
        return cls_outputs, box_outputs

class CustomSpec(EfficientDetModelSpec):
    def create_model(self):
        return MyModel()
# TODO
# override tensorflow_examples.lite.model_maker.core.task.model_spec.object_detector_spec.EfficientDetModelSpec.create_model
# to return model with the same outputs as tensorflow_examples.lite.model_maker.third_party.efficientdet.keras.train_lib.EfficientDetNetTrainHub
# if that won't work you'll have to rewrite a half of tflite model maker in kers to run this...

In [ ]:
m = MyModel()
m.build((None, 224, 224, 3))
m.summary()

In [ ]:
model = object_detector.create(
    train_data,
    model_spec=CustomSpec('efficientdet-lite0', uri=""),
    batch_size=1,
    train_whole_model=True,
    # validation_data=validation_data,
        epochs=100,
        # do_train=False
)

In [ ]:
history = model.model.history.history

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['loss'])

In [ ]:
plt.plot(history['cls_loss'])

In [ ]:
plt.plot(history['box_loss'])

In [ ]:
from PIL import Image
import numpy as np
img = np.array(Image.new('RGB', (320,320)))
img = tf.image.convert_image_dtype(img, tf.float32)
img = img[tf.newaxis, :]

In [ ]:
model(img)

In [ ]:
# TODO convert model to saved_model first, then to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
converter.target_spec.supported_ops

In [ ]:
tflite_model = converter.convert()

with tf.io.gfile.GFile('custom.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:



config = {
    'name': 'efficientdet-lite0',
    'act_type': 'relu6',
    'image_size': (320, 320),
    'target_size': None,
    'input_rand_hflip': True,
    'jitter_min': 0.1,
    'jitter_max': 2.0,
    'autoaugment_policy': None,
    'grid_mask': False,
    'sample_image': None,
    'map_freq': 5,
    'num_classes': 2,
    'seg_num_classes': 3,
    'heads': ['object_detection'],
    'skip_crowd_during_training': True,
    'label_map': {1: 'watch-face'},
    'max_instances_per_image': 100,
    'regenerate_source_id': False,
    'min_level': 3,
    'max_level': 7,
    'num_scales': 3,
    'aspect_ratios': [1.0, 2.0, 0.5],
    'anchor_scale': 3.0,
    'is_training_bn': True,
    'momentum': 0.9,
    'optimizer': 'sgd',
    'learning_rate': 0.08,
    'lr_warmup_init': 0.008,
    'lr_warmup_epoch': 1.0,
    'first_lr_drop_epoch': 200.0,
    'second_lr_drop_epoch': 250.0,
    'poly_lr_power': 0.9,
    'clip_gradients_norm': 10.0,
    'num_epochs': 1,
    'data_format': 'channels_last',
    'mean_rgb': 127.0,
    'stddev_rgb': 128.0,
    'scale_range': False,
    'label_smoothing': 0.0,
    'alpha': 0.25,
    'gamma': 1.5,
    'delta': 0.1,
    'box_loss_weight': 50.0,
    'iou_loss_type': None,
    'iou_loss_weight': 1.0,
    'weight_decay': 4e-05,
    'strategy': None,
    'mixed_precision': False,
    'loss_scale': None,
    'model_optimizations': {},
    'box_class_repeats': 3,
    'fpn_cell_repeats': 3,
    'fpn_num_filters': 64,
    'separable_conv': True,
    'apply_bn_for_resampling': True,
    'conv_after_downsample': False,
    'conv_bn_act_pattern': False,
    'drop_remainder': True,
    'nms_configs': {
        'method': 'gaussian', 'iou_thresh': None, 'score_thresh': 0.0, 'sigma': None,
        'pyfunc': False, 'max_nms_inputs': 0, 'max_output_size': 100
    },
    'tflite_max_detections': 25,
    'fpn_name': None,
    'fpn_weight_method': 'sum',
    'fpn_config': None,
    'survival_prob': None,
    'img_summary_steps': None,
    'lr_decay_method': 'cosine',
    'moving_average_decay': 0,
    'ckpt_var_scope': None,
    'skip_mismatch': True,
    'backbone_name': 'efficientnet-lite0',
    'backbone_config': None,
    'var_freeze_expr': None,
    'use_keras_model': True,
    'dataset_type': None,
    'positives_momentum': None,
    'grad_checkpoint': False,
    'verbose': 0,
    'save_freq': 'epoch',
    'profile': False,
    'model_name': 'efficientdet-lite0',
    'steps_per_execution': 1,
    'model_dir': '/var/folders/ym/8q_z70bx4hb86r1g8_5_98kw0000gn/T/tmpp34ys9s7',
    'batch_size': 8,
    'tf_random_seed': 111111,
    'debug': False
}